In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from app.database import SessionLocal, Base, engine
from app.models import *
from app import models
from app.utils import hash_password, reset_db
import requests
from datetime import datetime

import random
import uuid
from typing import List, Dict, Set, Tuple

import numpy as np
from faker import Faker
from app.models import (
    User,
    Group,
    GroupMembership,
    Report,
    Contest,
    ContestParticipation,
    Role,
)

In [2]:
def getContest(cid):
    url = 'https://codeforces.com/api/contest.standings'
    params = {
        'contestId': cid
    }
    res = requests.get(url, params).json()
    if res['status'] != 'OK':
        return None
    res = res['result']
    return {
        'contest': res['contest'],
        'standings': [
            {
                'handle': i['party']['members'][0]['handle'],
                'rank': i['rank'],
                'points': i['points'],
                'penalty': i['penalty']
            } for i in res['rows']
        ]
    }

def random_subset(n, k):
    return random.sample([i for i in range(n)], k)

def pick_random_subset(population: list, k: int) -> list:
    """k distinct elements from population (k may be 0)."""
    if k == 0:
        return []
    return random.sample(population, k)

def get_mock_contest_standings(cid, include_users=None):
    if include_users is None:
        include_users = []
    res = {
        'contest': {
            'id': cid,
            'name': f'Codeforces Round {cid}',
            'type': 'CF',
            'phase': 'FINISHED',
            'frozen': False,
            'durationSeconds': 7200,
            'startTimeSeconds': 1746110100,
            'relativeTimeSeconds': 267545
        },
        'standings': [
        ]
    }
    num_participants = 8000
    standings = [
        {'handle': 'random-user'+str(i+1), 'rank': (i+1), 'points': 9754 - i, 'penalty': 0}
        for i in range(num_participants)
    ]
    res['standings'] = standings
    udx = pick_random_subset([i for i in range(num_participants)], len(include_users))

    for i in range(len(udx)):
        res['standings'][udx[i]]['handle'] = include_users[i]
    
    return res


In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
Faker.seed(SEED)
faker = Faker()

NUM_USERS = 5_000
NUM_GROUPS = 30
NUM_CONTESTS = 5
NUM_PARTICIPATIONS = 15_000
NUM_REPORTS = 100
NUM_ANNOUNCEMENTS = 30

DEFAULT_PASS = "devpass"
PAST_FIVE_CONTESTS = ['2101', '2102', '2107', '2108', '2104']

In [4]:
sample_set = set()
for contest in PAST_FIVE_CONTESTS:
    standings = getContest(contest)
    for i in range(min(2000, len(standings['standings']))):
        row = standings['standings'][i]
        sample_set.add(row['handle'])

len(sample_set)

6116

In [5]:
reset_db()
Base.metadata.create_all(bind=engine)

db = SessionLocal()

dropping all tables...
all tables dropped.
creating tables from models...
schema rebuilt.


In [6]:
# POPULATE USERS - add corresponding codeforces handles that are ACTUALLY present on cf - will make subsequent testing easier
# create admins and common group
admin_users = [
    User(
        user_id="shrey",
        role=Role.admin,
        cf_handle="negative-xp",
        atcoder_handle=None,
        codechef_handle=None,
        twitter_handle=None,
        trusted_score=88,
        hashed_password=hash_password(DEFAULT_PASS),
    ),
    User(
        user_id="ani",
        role=Role.admin,
        cf_handle="roomTemperatureIQ",
        atcoder_handle=None,
        codechef_handle=None,
        twitter_handle=None,
        trusted_score=88,
        hashed_password=hash_password(DEFAULT_PASS),
    ),
]

users = admin_users

# create users mappted to cf handles

ii = 0
for handle in sample_set:
    uid = 'testUser' + str(len(users)-1)
    users.append(
        User(
            user_id=uid,
            role=Role.user,
            cf_handle=handle,
            atcoder_handle=None if random.random() < 0.5 else uid + "_ac",
            codechef_handle=None if random.random() < 0.7 else uid + "_cc",
            twitter_handle=None if random.random() < 0.6 else uid + "_tw",
            trusted_score=random.randint(0, 100),
            hashed_password=hash_password(DEFAULT_PASS),
        )
    )
    
db.add_all(users)
db.commit()

user_df = pd.read_sql("SELECT * FROM users", engine)
print(user_df.shape)
user_df.iloc[0:20]

(6118, 10)


,user_id,role,create_date,cf_handle,atcoder_handle,codechef_handle,twitter_handle,internal_default_rated,trusted_score,hashed_password
0,shrey,admin,2025-05-12 05:16:53.767940,negative-xp,None,None,None,True,88,yqjw8aG5kBAMpQdhsSZG5O8T3kKDIDfTsHCN+MlGKthw0R...
1,ani,admin,2025-05-12 05:16:53.767940,roomTemperatureIQ,None,None,None,True,88,s0FSvBCe5VFoB+toXlX3unzmsbRPTnxdVcXKnNhQfUYjUu...
2,testUser1,user,2025-05-12 05:16:53.767940,ishmeal,testUser1_ac,None,None,True,28,KWo6jIRAx6O5CZ8eLFw9ZVM/j3ggo8mZhiHDhAKmTPtRN3...
3,testUser2,user,2025-05-12 05:16:53.767940,Max_Gameversion,None,None,testUser2_tw,True,69,i8GCPbhwk0CCfgFX5wwlSrOwd7U+LeDqevY9F5IxVDdydw...
4,testUser3,user,2025-05-12 05:16:53.767940,19.30,None,None,None,True,27,Bxk2LLzJI/akFdCnVruahQ3gxOMuI6/4OnmT0cfcKOQ3vK...
5,testUser4,user,2025-05-12 05:16:53.767940,Typedef_Nan,None,None,None,True,91,UaPEE5Pt8Mkhi/dkMHgDSqGy23r6IeakQ3zTN27BOKPZFF...
6,testUser5,user,2025-05-12 05:16:53.767940,morshaline_1,testUser5_ac,None,None,True,75,3SsgwsdwVfHhN/C4cJP5G33nLln26vL9Lbp5lgCCBa0Qx9...
7,testUser6,user,2025-05-12 05:16:53.767940,Arsh_Chand,None,testUser6_cc,testUser6_tw,True,20,7aSMuBiK/TYdoZCPM0Jmd73VvaZo7hEsDF1R0YBfavJPp3...
8,testUser7,user,2025-05-12 05:16:53.767940,yulik.daniel,testUser7_ac,None,None,True,97,uiKUHjAes5WIVU5T9gRfwMoRGDShnwCk2FfFycdN0mTtfq...
9,testUser8,user,2025-05-12 05:16:53.767940,gan_yu,None,None,None,True,44,8M6ckXZcW2PeaXvBiRdWo4RycjtaFNNbBxikqfMbTymeIr...


In [7]:
# let's just simulate contests in the main group for now uwux

common_group = Group(
    group_id="main",
    group_name="main",
    group_description="group consisting of ALL users",
    is_private=False,
)

groups = [common_group]
# for g_idx in range(1, NUM_GROUPS):
#     g_id = f"g{g_idx:02d}"
#     g_name = faker.unique.catch_phrase().lower().replace(" ", "‑")
#     is_priv = random.random() < 0.3
#     group = Group(
#         group_id=g_id,
#         group_name=g_name,
#         group_description=faker.sentence(nb_words=10),
#         is_private=is_priv,
#     )
#     groups.append(group)

db.add_all(groups)
db.commit()

group_df = pd.read_sql("SELECT * FROM groups", engine)
print(group_df.shape)
group_df.head()

(1, 5)


,group_id,group_name,group_description,is_private,create_date
0,main,main,group consisting of ALL users,False,2025-05-12 05:17:05.135725


In [8]:
# POPULATE MEMBERSHIPS
memberships = []


# common group should have ALL users
memberships.append(GroupMembership(
    user_id=users[0].user_id,
    group_id=groups[0].group_id,
    role=Role.admin,
    user_group_rating=1500,
    user_group_max_rating=1500,
))

for i in range(1, len(users)):
    memberships.append(
        GroupMembership(
            user_id=users[i].user_id,
            group_id=groups[0].group_id,
            role=Role.user,
            user_group_rating=1500,
            user_group_max_rating=1500,
        )
    )
db.add_all(memberships)
db.commit()

membership_df = pd.read_sql("SELECT * FROM group_memberships", engine)
print(membership_df.shape)
membership_df.head()

(6118, 6)


,user_id,group_id,role,user_group_rating,user_group_max_rating,status
0,shrey,main,admin,1500,1500,active
1,ani,main,user,1500,1500,active
2,testUser1,main,user,1500,1500,active
3,testUser2,main,user,1500,1500,active
4,testUser3,main,user,1500,1500,active


In [10]:
# create a few contest objects - 

def get_contest_object(contest_cf_id):
    cf_json = getContest(contest_cf_id)
    start_time = datetime.fromtimestamp(int(cf_json['contest']['startTimeSeconds']))
    duration_seconds = int(cf_json['contest']['durationSeconds'])
    contest_name = cf_json['contest']['name']

    return Contest(
            contest_id = "cid_" + str(int(random.random() * 100000)),
            cf_contest_id = contest,
            cf_standings = getContest(contest),
            finished = True,
            start_time=start_time,
            duration_seconds=duration_seconds,
            contest_name=contest_name
        )

    
contests = []

for contest in PAST_FIVE_CONTESTS:
    contests.append(
        get_contest_object(contest)
    )

In [12]:
len(contests[0].__dict__['cf_standings']['standings'])

859

In [13]:
db.add_all(contests)
db.commit()

In [14]:
contests_df = pd.read_sql("SELECT * FROM contests", engine)
contests_df.head()

,contest_id,cf_contest_id,cf_standings,finished,start_time,duration_seconds,contest_name
0,cid_88168,2101,"{'contest': {'id': 2101, 'name': 'Codeforces R...",True,2025-05-11 20:05:00,9000,Codeforces Round 1024 (Div. 1)
1,cid_14011,2102,"{'contest': {'id': 2102, 'name': 'Codeforces R...",True,2025-05-11 20:05:00,9000,Codeforces Round 1024 (Div. 2)
2,cid_80900,2107,"{'contest': {'id': 2107, 'name': 'Codeforces R...",True,2025-05-05 20:05:00,8100,Codeforces Round 1023 (Div. 2)
3,cid_81355,2108,"{'contest': {'id': 2108, 'name': 'Codeforces R...",True,2025-05-01 20:05:00,7200,Codeforces Round 1022 (Div. 2)
4,cid_11004,2104,"{'contest': {'id': 2104, 'name': 'Educational ...",True,2025-04-28 20:05:00,7200,Educational Codeforces Round 178 (Rated for Di...


In [25]:
def randomly_generate_contest_participations(cid, gid):
    # return a list of contest_participation objects given a cid and a gid
    contest = db.query(Contest).get(cid)
    group = db.query(Group).get(gid)

    if not contest or not group:
        return f"either contest {cid} or group {gid} does not exist"

    N = len(group.memberships)
    pids = random_subset(N, N//2)
    res = []

    for i in pids:
        res.append(
            ContestParticipation(
                user_id = group.memberships[i].user_id,
                group_id = gid,
                contest_id = cid,
                rating_before = group.memberships[i].user_group_rating
            )
        )
    return res

    

In [31]:
contest0 = randomly_generate_contest_participations(contests[0].contest_id, groups[0].group_id)

/tmp/ipykernel_9542/1121266973.py:3: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  contest = db.query(Contest).get(cid)
/tmp/ipykernel_9542/1121266973.py:4: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  group = db.query(Group).get(gid)


In [40]:
# simulate contest 1 from start to finish - including rating changes

cur_contest = contests[0]
cur_group = groups[0]

# contest is added to our db through a cron jon
# users start registering

registrations = randomly_generate_contest_participations(cur_contest.contest_id, cur_group.group_id)

db.add_all(registrations)
db.commit()



/tmp/ipykernel_9542/1121266973.py:3: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  contest = db.query(Contest).get(cid)
/tmp/ipykernel_9542/1121266973.py:4: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  group = db.query(Group).get(gid)


In [53]:
# contest finishes - update contest participation objects for every group

# first update every contest participation object to include RANK



In [74]:
def update_participation_status(contest_id, group_id):
    registrations = db.query(ContestParticipation).filter_by(contest_id = contest_id, group_id=group_id).all()
    reg_set = set([db.get(User, i.user_id).cf_handle for i in registrations])
    standings = cur_contest.cf_standings['standings']
    relevant_standings = [i for i in standings if i['handle'] in reg_set]
    actual_rank = dict()
    for i in range(len(relevant_standings)):
        actual_rank[relevant_standings[i]['handle']] = (i+1)
        
    cf_handle_set = set([i['handle'] for i in relevant_standings])

    updated_regs = []
    for reg in registrations:
        cfh = db.get(User, reg.user_id).cf_handle
        if cfh not in cf_handle_set:
            reg.took_part = False
        else:
            reg.took_part = True
            reg.rank = actual_rank[cfh]
        updated_regs.append(reg)
    return updated_regs

    
            

In [76]:
updated_regs = update_participation_status(cur_contest.contest_id, cur_group.group_id)
db.add_all(updated_regs)
db.commit()

In [72]:
# updated_regs

In [78]:
parts = db.query(ContestParticipation).filter_by(contest_id=cur_contest.contest_id, group_id=cur_group.group_id).all()
len(parts)

3059

In [26]:
groups[0].__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x75ab7c946150>,
 'group_id': 'main',
 'group_name': 'main',
 'is_private': False,
 'create_date': datetime.datetime(2025, 5, 12, 3, 12, 8, 359241),
 'group_description': 'group consisting of ALL users'}